In [1]:
#do what the keras people suggest

from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
plt.style.use('demo_style.mplstyle')

#tf.keras.backend.clear_session()  # For easy reset of notebook state.


In [2]:
#get job info

import os

os.environ["SLURM_JOB_ID"]

'771048'

In [3]:
q003_data = np.load('/global/cscratch1/sd/stephey/march2020_data/Output/q003_data.npy')
#q003_max = np.load('/global/cscratch1/sd/stephey/march2020_data/Output/q003_max.npy')
#q003_norm = np.load('/global/cscratch1/sd/stephey/march2020_data/Output/q003_norm.npy')

q103_data = np.load('/global/cscratch1/sd/stephey/march2020_data/Output2/q103_data.npy') 
#q103_max = np.load('/global/cscratch1/sd/stephey/march2020_data/Output2/q103_max.npy')
#q103_norm = np.load('/global/cscratch1/sd/stephey/march2020_data/Output2/q103_norm.npy')

#add in our post-quench data
#need to cut that down to size, too

pq_data = np.load('/global/cscratch1/sd/stephey/march2020_data/Q3_Q103_postquench/pq_data.npy')

In [4]:
#cut to make square for later autoencoder friendly size

q003_data = q003_data[:,0:512,:]
q103_data = q103_data[:,0:512,:]
pq_data = pq_data[:,0:512,:]


In [5]:


#we shuffle in the training so it's ok
qall_data = np.concatenate((q003_data, q103_data, pq_data), axis=0)

print(qall_data.shape)


(9807, 512, 512)


In [6]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, Activation, MaxPool2D
from tensorflow.keras.layers import BatchNormalization, Flatten, Reshape, Conv2DTranspose, LeakyReLU
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD


In [7]:

autoencoder = tf.keras.models.load_model('conv2d_autoencoder')

autoencoder.summary()

#this seems either buggy or we've done something wrong



Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 512, 512, 1)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 256, 256, 32)      1600      
_________________________________________________________________
batch_normalization (BatchNo (None, 256, 256, 32)      128       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 256, 256, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 32)      50208     
_________________________________________________________________
batch_normalization_1 (Batch (None, 128, 128, 32)      128       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 128, 128, 32)      0     

In [8]:
encoder = tf.keras.Model(inputs=autoencoder.inputs, outputs=autoencoder.get_layer('latent').output)
encoder.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 512, 512, 1)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 256, 256, 32)      1600      
_________________________________________________________________
batch_normalization (BatchNo (None, 256, 256, 32)      128       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 256, 256, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 32)      50208     
_________________________________________________________________
batch_normalization_1 (Batch (None, 128, 128, 32)      128       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 128, 128, 32)      0     

In [ ]:
#put all our data into the encoder

qall_reshape = np.expand_dims(qall_data, -1)
encoded_all = encoder.predict(qall_reshape)

np.save('encoded_all', encoded_all)